In [ ]:

import openai
import os
from openai import OpenAI
from agents import generator
from agents import evaluator
import gradio as gr
import time
import pandas as pd
openai.api_key = os.getenv('OPENAI_API_KEY')


In [2]:
# Define input components

dish_name_input = gr.Textbox(
    lines=1,
    placeholder='Enter dish name here',
    label='Dish Name'
)

original_recipe_input = gr.Textbox(
    lines=15,
    placeholder='Enter the original recipe here...(leave it empty if you do not have one)',
    label='Original Recipe'
)

restrictions_input = gr.Textbox(
    lines=5,
    placeholder='Enter any dietary restrictions, allergies  or any specifications here...',
    label='User Restrictions'
)

# Define output component
modified_recipe_output = gr.Textbox(
    label='New Recipe'
)


# Create the Gradio Interface
app = gr.Interface(
    fn=generator,
    inputs=[dish_name_input, original_recipe_input, restrictions_input],
    outputs= modified_recipe_output,
    title='Safe Plate',
    description='Provide a dish name or an original recipe along with your dietary restrictions. The app will generate a modified recipe that adheres to your restrictions while maintaining the original flavor.'
)


In [3]:
app.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [4]:
#some test cases
test_cases = [
    {
        'test_id': 1,
        'dish_name': 'Spaghetti Carbonara',
        'original_recipe': """
Spaghetti Carbonara

Ingredients:
- 200g spaghetti
- 100g pancetta
- 2 large eggs
- 50g pecorino cheese
- 50g parmesan cheese
- Freshly ground black pepper
- Salt

Instructions:
1. Boil water and cook spaghetti until al dente.
2. In a bowl, beat the eggs and mix in grated cheeses.
3. Fry pancetta until crispy.
4. Combine spaghetti with pancetta.
5. Remove from heat and quickly mix in egg and cheese mixture.
6. Serve immediately with extra cheese and pepper.
""",
        'restrictions': '- Allergic to eggs.\n- No pork products due to religious restrictions.'
    },
    {
        'test_id': 2,
        'dish_name': 'Chicken Alfredo',
        'original_recipe': """
Chicken Alfredo

Ingredients:
- 200g fettuccine pasta
- 2 chicken breasts
- 1 cup heavy cream
- 1/2 cup butter
- 1 cup grated parmesan cheese
- 2 cloves garlic, minced
- Salt and pepper to taste
- Fresh parsley for garnish

Instructions:
1. Cook pasta according to package instructions.
2. Season chicken with salt and pepper, then cook until done. Slice into strips.
3. In a pan, melt butter and sauté garlic.
4. Add heavy cream and bring to a simmer.
5. Stir in parmesan cheese until sauce thickens.
6. Add cooked pasta and chicken to the sauce. Toss to combine.
7. Serve garnished with fresh parsley.
""",
        'restrictions': '- Lactose intolerant.\n- Gluten-free diet.'
    },
    {
        'test_id': 3,
        'dish_name': 'Beef Tacos',
        'original_recipe': """
Beef Tacos

Ingredients:
- 500g ground beef
- 1 packet taco seasoning
- 12 taco shells
- Lettuce, shredded
- Cheddar cheese, shredded
- Sour cream
- Salsa
- Olive oil

Instructions:
1. Heat oil in a pan and brown the ground beef.
2. Add taco seasoning and cook according to package instructions.
3. Warm taco shells in the oven.
4. Assemble tacos with beef, lettuce, cheese, sour cream, and salsa.
5. Serve immediately.
""",
        'restrictions': '- Vegetarian diet.\n- Allergic to dairy products.'
    },
]


In [7]:

# Initialize a list to store the results
results = []

for test in test_cases:
    test_id = test['test_id']
    dish_name = test['dish_name']
    original_recipe = test['original_recipe']
    restrictions = test['restrictions']
    
    print(f"Processing Test Case {test_id}: {dish_name}")
    
    # Generate the modified recipe
    modified_recipe = generator(dish_name, original_recipe, restrictions)
    
    # Evaluate the modified recipe
    evaluation = evaluator(original_recipe, modified_recipe, restrictions)
    
    # Append the results
    results.append({
        'Test ID': test_id,
        'Dish Name': dish_name,
        'Original Recipe': original_recipe,
        'User Restrictions': restrictions,
        'Modified Recipe': modified_recipe,
        'Valid': evaluation
    })
    
    time.sleep(1)

df_results = pd.DataFrame(results)



Processing Test Case 1: Spaghetti Carbonara
Processing Test Case 2: Chicken Alfredo
Processing Test Case 3: Beef Tacos


In [8]:
def accuracy(df_results):
    rate = df_results['Valid'].sum() / df_results.shape[0]
    return f"The accuracy of the Generator is {rate:.2%}"
accuracy(df_results)

'The accuracy of the Generator is 100.00%'